# API Call Stuff

In [148]:
from sodapy import Socrata
import pandas as pd
import geopandas as gpd
import plotly.express as px
import h3
import os
from dotenv import load_dotenv
from folium import Map, Marker, GeoJson
import folium
from geojson.feature import Feature, FeatureCollection
import geojson
import json
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from tqdm import tqdm

In [349]:

load_dotenv()

soda_token = os.getenv("SODA_TOKEN")

client = Socrata("data.cityofnewyork.us", app_token=soda_token)
client.timeout = 120

In [350]:
results = client.get("uacg-pexx", limit=500000)

results_df = pd.DataFrame.from_records(results)

In [351]:
results_df = results_df[(results_df['pickup_longitude'] != 0) & (results_df['pickup_latitude'] != 0)]

In [352]:
results_df.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,ratecodeid,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-02-19T19:26:14.000,2016-02-19T19:49:33.000,1,9.36,-73.98419189453125,40.767009735107422,1,N,-73.926651000976562,40.868301391601563,1,28,1,0.5,2.5,0,0.3,32.3
1,1,2016-02-05T18:00:30.000,2016-02-05T18:05:47.000,2,0.8,-73.964889526367187,40.769828796386719,1,N,-73.954902648925781,40.769577026367188,1,5.5,1,0.5,1.5,0,0.3,8.8
2,2,2016-02-21T01:16:58.000,2016-02-21T01:25:32.000,1,2.12,-73.921470642089844,40.755966186523438,1,N,-73.886528015136719,40.755386352539063,1,9,0.5,0.5,0,0,0.3,10.3
3,2,2016-02-27T13:50:17.000,2016-02-27T13:56:02.000,1,0.5,-73.991737365722656,40.729568481445313,1,N,-73.9874267578125,40.733741760253906,1,5.5,0,0.5,1.26,0,0.3,7.56
4,2,2016-02-05T21:01:45.000,2016-02-05T21:27:48.000,1,4.78,-73.951576232910156,40.778469085693359,1,N,-73.998527526855469,40.729530334472656,1,19.5,0.5,0.5,4.16,0,0.3,24.96


# Visualizations
KDE over hour, day, and month of pickups across Manhattan

In [353]:
results_df['tpep_pickup_datetime'] = pd.to_datetime(results_df['tpep_pickup_datetime'])
results_df['tpep_dropoff_datetime'] = pd.to_datetime(results_df['tpep_dropoff_datetime'])

results_df['pickup_hour'] = results_df['tpep_pickup_datetime'].dt.hour
results_df['pickup_day'] = results_df['tpep_pickup_datetime'].dt.day_of_week
results_df['pickup_month'] = results_df['tpep_pickup_datetime'].dt.month
results_df['date'] = results_df['tpep_pickup_datetime'].dt.date

In [ ]:
fig = px.density_map(
    results_df,
    lat='pickup_latitude',
    lon='pickup_longitude',
    animation_frame='pickup_hour',
    category_orders={'pickup_hour': sorted(results_df['pickup_hour'].unique())},
    radius=2,
    z=None,
    center={'lat': 40.73, 'lon': -74.0},
    zoom=10.5,
    map_style='carto-positron',
    title='Density of Pickup Locations By Hour',
    height=800
)

fig.show()

In [ ]:
fig = px.density_map(
    results_df,
    lat='pickup_latitude',
    lon='pickup_longitude',
    animation_frame='pickup_day',
    category_orders={'pickup_day': sorted(results_df['pickup_day'].unique())},
    radius=2,
    z=None,
    center={'lat': 40.73, 'lon': -74.0},
    zoom=10.5,
    map_style='carto-positron',
    title='Density of Pickup Locations By Day',
    height=800
)

fig.show()

In [ ]:
fig = px.density_map(
    results_df,
    lat='pickup_latitude',
    lon='pickup_longitude',
    animation_frame='pickup_month',
    category_orders={'pickup_month': sorted(results_df['pickup_month'].unique())},
    radius=2,
    z=None,
    center={'lat': 40.73, 'lon': -74.0},
    zoom=10.5,
    map_style='carto-positron',
    title='Density of Pickup Locations By Month',
    height=800
)

fig.show()

# Hex Binning
The KDEs didn't really allow us to distinguish the peak values the way that I would want them to, so lets use a hexbinning method, which would also allow us to visualize frequencies and also cluster over a discrete set of geographies.

For each of the dataframes, I will be translating each of the longitude latitude pairs into their respective hex cells, and calculating the average number of trips from each of the cell ids and between each of the cell-id pairs by hour, day of week, and month.

In [80]:
hr_average_pu = {}
hr_average_do = {}
hr_average_pu_do = {}

day_average_pu = {}
day_average_do = {}
day_average_pu_do = {}

month_average_pu = {}
month_average_do = {}
month_average_pu_do = {}

In [ ]:
results_df['pickup_latitude'] = results_df['pickup_latitude'].astype(float)
results_df['pickup_longitude'] = results_df['pickup_longitude'].astype(float)
results_df['dropoff_latitude'] = results_df['dropoff_latitude'].astype(float)
results_df['dropoff_longitude'] = results_df['dropoff_longitude'].astype(float)

In [88]:
def merge_pu_do_dicts(time_average_d: dict, slice_times: pd.Series) -> dict:
    for period in slice_times.index.get_level_values(0).unique():
        time_average_d[period] = time_average_d.get(period, pd.Series(dtype=float)).add(slice_times[period], fill_value=0)

    return time_average_d

In [ ]:
results_df['pu_cell_id'] = results_df.apply(lambda row: h3.latlng_to_cell(
    lat=row['pickup_latitude'],
    lng=row['pickup_longitude'],
    res=10
), axis=1)

results_df['do_cell_id'] = results_df.apply(lambda row: h3.latlng_to_cell(
    lat=row['dropoff_latitude'],
    lng=row['dropoff_longitude'],
    res=10
), axis=1)
results_df['pu_do_ids'] = results_df['pu_cell_id'].str.cat(results_df['do_cell_id'], sep="_")

In [ ]:
pu_cell_id_by_hr = results_df.groupby('pickup_hour')['pu_cell_id'].value_counts()
pu_cell_id_by_day = results_df.groupby('pickup_day')['pu_cell_id'].value_counts()
pu_cell_id_by_month = results_df.groupby('pickup_month')['pu_cell_id'].value_counts()

do_cell_id_by_hr = results_df.groupby('pickup_hour')['do_cell_id'].value_counts()
do_cell_id_by_day = results_df.groupby('pickup_day')['do_cell_id'].value_counts()
do_cell_id_by_month = results_df.groupby('pickup_month')['do_cell_id'].value_counts()

pu_do_id_by_hr = results_df.groupby('pickup_hour')['pu_do_ids'].value_counts()
pu_do_id_by_day = results_df.groupby('pickup_day')['pu_do_ids'].value_counts()
pu_do_id_by_month = results_df.groupby('pickup_month')['pu_do_ids'].value_counts()

In [ ]:
hr_average_pu = merge_pu_do_dicts(hr_average_pu, pu_cell_id_by_hr)
day_average_pu = merge_pu_do_dicts(day_average_pu, pu_cell_id_by_day)
month_average_pu = merge_pu_do_dicts(month_average_pu, pu_cell_id_by_month)

hr_average_do = merge_pu_do_dicts(hr_average_do, do_cell_id_by_hr)
day_average_do = merge_pu_do_dicts(day_average_do, do_cell_id_by_day)
month_average_do = merge_pu_do_dicts(month_average_do, do_cell_id_by_month)

hr_average_pu_do = merge_pu_do_dicts(hr_average_pu_do, pu_do_id_by_hr)
day_average_pu_do = merge_pu_do_dicts(day_average_pu_do, pu_do_id_by_day)
month_average_pu_do = merge_pu_do_dicts(month_average_pu_do, pu_do_id_by_month)

summing pu_cell_id
8a2a100122b7fff      1.0
8a2a10013177fff      1.0
8a2a100136d7fff      1.0
8a2a10013adffff      1.0
8a2a10015c77fff      1.0
                   ...  
8a2a10775917fff      1.0
8a2a10775a57fff      1.0
8a2a10775ac7fff      1.0
8a2a10776c2ffff      1.0
8a754e64992ffff    275.0
Length: 2505, dtype: float64 and pickup_hour  pu_cell_id     
0            8a754e64992ffff    275
             8a2a103b1c97fff    144
             8a2a10721b1ffff     92
             8a2a1072caa7fff     83
             8a2a100d2c17fff     80
                               ... 
             8a2a1077590ffff      1
             8a2a10775917fff      1
             8a2a10775a57fff      1
             8a2a10775ac7fff      1
             8a2a10776c2ffff      1
Name: count, Length: 2505, dtype: int64
summing pu_cell_id
8a2a10012487fff      1.0
8a2a1001c49ffff      1.0
8a2a1001e807fff      1.0
8a2a100367b7fff      1.0
8a2a1005260ffff      1.0
                   ...  
8a2a10d592dffff      1.0
8a2a12b6cacfff

## Averaging Traffic by Hex Bin
In order to establish the "average" number of pickups, dropoffs, and pickup/dropoff pairs at a given hour, we need to be able to divide by the number of unique hours/days/months that appear in the dataset.

In [269]:
hr_average_pu = {}
hr_average_do = {}
hr_average_pu_do = {}

day_average_pu = {}
day_average_do = {}
day_average_pu_do = {}

month_average_pu = {}
month_average_do = {}
month_average_pu_do = {}

In [270]:
total_items = int(client.get_metadata("uacg-pexx")['columns'][0]['cachedContents']['count'])

In [271]:
PAGE_SIZE = 300000
dates = set()
months = set()
curr_offset = 0
valid_trips = 0
selected_columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
                    'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

selected_string = ", ".join(selected_columns)
pbar = tqdm(total = total_items, desc="Fetching Records", unit="records")
while True:
    items = client.get("uacg-pexx", limit=PAGE_SIZE, offset = curr_offset, select = selected_string)
    items_df = pd.DataFrame.from_records(items)

    if items_df.shape[0] == 0:
        break

    curr_offset += items_df.shape[0]
    pbar.update(items_df.shape[0])

    if 'pickup_latitude' not in items_df.columns:
        continue
    
    items_df['pickup_latitude'] = items_df['pickup_latitude'].astype(float)
    items_df['pickup_longitude'] = items_df['pickup_longitude'].astype(float)
    items_df['dropoff_latitude'] = items_df['dropoff_latitude'].astype(float)
    items_df['dropoff_longitude'] = items_df['dropoff_longitude'].astype(float)
    items_df = items_df[~(items_df['pickup_latitude'] == 0) & ~(items_df['dropoff_latitude'] == 0) &
                        ~(items_df['pickup_latitude'].isna()) & ~(items_df['dropoff_latitude'].isna())]

    valid_trips += items_df.shape[0]

    items_df['tpep_pickup_datetime'] = pd.to_datetime(items_df['tpep_pickup_datetime'])
    items_df['tpep_dropoff_datetime'] = pd.to_datetime(items_df['tpep_dropoff_datetime'])

    items_df['pickup_hour'] = items_df['tpep_pickup_datetime'].dt.hour
    items_df['pickup_day'] = items_df['tpep_pickup_datetime'].dt.day_of_week
    items_df['pickup_month'] = items_df['tpep_pickup_datetime'].dt.month
    items_df['pickup_date'] = items_df['tpep_pickup_datetime'].dt.date

    items_df['pu_cell_id'] = items_df.apply(lambda row: h3.latlng_to_cell(
        lat=row['pickup_latitude'],
        lng=row['pickup_longitude'],
        res=10
    ), axis=1)

    items_df['do_cell_id'] = items_df.apply(lambda row: h3.latlng_to_cell(
        lat=row['dropoff_latitude'],
        lng=row['dropoff_longitude'],
        res=10
    ), axis=1)
    items_df['pu_do_ids'] = items_df['pu_cell_id'].str.cat(items_df['do_cell_id'], sep="_")

    pu_cell_id_by_hr = items_df.groupby('pickup_hour')['pu_cell_id'].value_counts()
    pu_cell_id_by_day = items_df.groupby('pickup_day')['pu_cell_id'].value_counts()
    pu_cell_id_by_month = items_df.groupby('pickup_month')['pu_cell_id'].value_counts()

    do_cell_id_by_hr = items_df.groupby('pickup_hour')['do_cell_id'].value_counts()
    do_cell_id_by_day = items_df.groupby('pickup_day')['do_cell_id'].value_counts()
    do_cell_id_by_month = items_df.groupby('pickup_month')['do_cell_id'].value_counts()

    pu_do_id_by_hr = items_df.groupby('pickup_hour')['pu_do_ids'].value_counts()
    pu_do_id_by_day = items_df.groupby('pickup_day')['pu_do_ids'].value_counts()
    pu_do_id_by_month = items_df.groupby('pickup_month')['pu_do_ids'].value_counts()

    hr_average_pu = merge_pu_do_dicts(hr_average_pu, pu_cell_id_by_hr)
    day_average_pu = merge_pu_do_dicts(day_average_pu, pu_cell_id_by_day)
    month_average_pu = merge_pu_do_dicts(month_average_pu, pu_cell_id_by_month)

    hr_average_do = merge_pu_do_dicts(hr_average_do, do_cell_id_by_hr)
    day_average_do = merge_pu_do_dicts(day_average_do, do_cell_id_by_day)
    month_average_do = merge_pu_do_dicts(month_average_do, do_cell_id_by_month)

    hr_average_pu_do = merge_pu_do_dicts(hr_average_pu_do, pu_do_id_by_hr)
    day_average_pu_do = merge_pu_do_dicts(day_average_pu_do, pu_do_id_by_day)
    month_average_pu_do = merge_pu_do_dicts(month_average_pu_do, pu_do_id_by_month)

    dates.update(items_df['pickup_date'].unique())
    months.update(items_df['pickup_month'].unique())

pbar.close()

Fetching Records:   0%|          | 0/131165043 [00:19<?, ?records/s]


ReadTimeout: HTTPSConnectionPool(host='data.cityofnewyork.us', port=443): Read timed out. (read timeout=120)

In [274]:
print(valid_trips)
print(valid_trips / curr_offset * 100)

50317751
55.17297258771929


In [275]:
curr_offset

91200000

## Mapping
After calculating and aggregating trip counts by the hex cells, I will go ahead and visualize the hex bins in choropleth form, normalized by the highest and lowest values present in any of the time buckets of a given type (i.e. day, month, hour)

In [355]:
# This cell justs tests retrieving and saving a single cell at the chosen resolution

RESOLUTION = 9

h = h3.latlng_to_cell(lat = 40.73, lng = -74.0, res=RESOLUTION)

h_geom = h3.cells_to_geo(cells = [h])

hex_bin = {"res": RESOLUTION, "geometry": h_geom}

map_test = Map(location = [40.73, -74.0],
                  zoom_start = 10.5,
                  tiles = "cartodbpositron",
                  attr = '''© <a href="http://www.openstreetmap.org/copyright">
                          OpenStreetMap</a>contributors ©
                          <a href="http://cartodb.com/attributions#basemaps">
                          CartoDB</a>'''
                  )


hex_feature = Feature(geometry = hex_bin["geometry"],
                    id = 1,
                    properties = {"resolution": int(hex_bin["res"])})

geojson_result = json.dumps(hex_feature)


GeoJson(
        geojson_result,
        style_function = lambda feature: {
            'fillColor': None,
            'color': "green",
            'weight': 2,
            'fillOpacity': 0.05
        },
        name = "Example"
    ).add_to(map_test)

map_test.save(f'maps/map_test_{RESOLUTION}.html')

In [277]:
nyc_boundary = None
with open("geo/nyc_boroughs.json") as geo:
    nyc_boundary = geojson.load(geo)

In [357]:
boro_cells = {} # dictionary of boroughs to sets of hex cells
all_cells = []
for i in range(len(nyc_boundary['features'])):
    i_boro_cells = h3.geo_to_cells(nyc_boundary['features'][i]['geometry'], res=RESOLUTION)
    boro_id = nyc_boundary['features'][i]['properties']['BoroName']
    boro_cells[boro_id] = i_boro_cells
    all_cells.extend(i_boro_cells)

boro_feature_colls = {}
all_features = []
for id, cells in boro_cells.items():
    feature_list = []
    for cell in cells:
        cell_geom = [[lon, lat] for lat, lon in h3.cell_to_boundary(cell)]
        cell_geom.append(cell_geom[0])
        cell_feature = Feature(id=cell, geometry={'type': 'Polygon', 'coordinates': [cell_geom]})
        feature_list.append(cell_feature)

    feature_col = FeatureCollection(feature_list)
    all_features.extend(feature_list)
    boro_feature_colls[id] = feature_col

all_cells_feature_coll = FeatureCollection(all_features)

In [279]:
map_boros = Map(location = [40.73, -74.0],
                  zoom_start = 10.5,
                  tiles = "cartodbpositron",
                  attr = '''© <a href="http://www.openstreetmap.org/copyright">
                          OpenStreetMap</a>contributors ©
                          <a href="http://cartodb.com/attributions#basemaps">
                          CartoDB</a>'''
                  )

GeoJson(nyc_boundary,
        style_function = lambda feature: {
            'fillColor': None,
            'color': "green",
            'weight': 2
        },
        name = "NYC Boroughs"
        ).add_to(map_boros)
map_boros.save(f'maps/map_boros.html')

In [359]:
map_hex = Map(location = [40.73, -74.0],
                  zoom_start = 10.5,
                  tiles = "cartodbpositron",
                  attr = '''© <a href="http://www.openstreetmap.org/copyright">
                          OpenStreetMap</a>contributors ©
                          <a href="http://cartodb.com/attributions#basemaps">
                          CartoDB</a>'''
                  )

GeoJson(all_cells_feature_coll,
    style_function = lambda feature: {
        'fillColor': None,
        'color': "gray",
        'weight': 0.5,
        'fillOpacity': 0.1,
    },
    name = "Hex(10) NYC"
).add_to(map_hex)
folium.LayerControl().add_to(map_hex)
map_hex.save(f'maps/map_hex{RESOLUTION}.html')

In [291]:
num_days = len(dates)

In [305]:
hr_average_pu_avg = {key : (value / num_days) for key,value in hr_average_pu.items()}
hr_average_do_avg = {key : (value / num_days) for key,value in hr_average_do.items()}
hr_average_pu_do_avg = {key : (value / num_days) for key,value in hr_average_pu_do.items()}

In [311]:
min_pu_hr = min([hr_average_pu_avg[key].min() for key in hr_average_pu_avg.keys()])
max_pu_hr = max([hr_average_pu_avg[key].max() for key in hr_average_pu_avg.keys()])

min_do_hr = min([hr_average_do_avg[key].min() for key in hr_average_do_avg.keys()])
max_do_hr = max([hr_average_do_avg[key].max() for key in hr_average_do_avg.keys()])

min_point_hr = min(min_pu_hr, min_do_hr)
max_point_hr = max(max_pu_hr, max_do_hr)

In [313]:
max_point_hr

190.15934065934067

In [ ]:
import branca.colormap as bcm

HR_MIN = 0
HR_MAX = 200

hr_cmap = bcm.linear.YlOrRd_03.scale(vmin=HR_MIN, vmax=HR_MAX)
hr_cmap.caption = "Average Trips in Given Hour"

In [ ]:
map_choropleth = Map(location = [40.73, -74.0],
                  zoom_start = 10.5,
                  tiles = "cartodbpositron",
                  attr = '''© <a href="http://www.openstreetmap.org/copyright">
                          OpenStreetMap</a>contributors ©
                          <a href="http://cartodb.com/attributions#basemaps">
                          CartoDB</a>''',
                    prefer_canvas=True
                  )

for key in hr_average_pu_avg:
    subset = [feature for feature in all_cells_feature_coll['features'] if feature.id in hr_average_pu_avg[key]]
    GeoJson(FeatureCollection(subset),
        style_function = lambda feature, d=hr_average_pu_avg[key]: {
            'fillColor': hr_cmap(d[feature.id]),
            'weight': 0.05,
            'fillOpacity': 0.8,
        },
        show = False,
        name = f"Hour {key}"
    ).add_to(map_choropleth)

hr_cmap.add_to(map_choropleth)
folium.LayerControl().add_to(map_choropleth)
map_choropleth.save(f'maps/map_pickup{RESOLUTION}.html')

In [347]:
map_dropoff = Map(location = [40.73, -74.0],
                  zoom_start = 10.5,
                  tiles = "cartodbpositron",
                  attr = '''© <a href="http://www.openstreetmap.org/copyright">
                          OpenStreetMap</a>contributors ©
                          <a href="http://cartodb.com/attributions#basemaps">
                          CartoDB</a>''',
                    prefer_canvas=True
                  )

for key in hr_average_do_avg:
    subset = [feature for feature in all_cells_feature_coll['features'] if feature.id in hr_average_do_avg[key]]
    GeoJson(FeatureCollection(subset),
        style_function = lambda feature, d=hr_average_do_avg[key]: {
            'fillColor': hr_cmap(d[feature.id]),
            'weight': 0.05,
            'fillOpacity': 0.8,
        },
        show = False,
        name = f"Hour {key}"
    ).add_to(map_dropoff)

hr_cmap.add_to(map_dropoff)
folium.LayerControl().add_to(map_dropoff)
map_dropoff.save(f'maps/map_dropoff{RESOLUTION}.html')

# Other Things to Visualize and Assume
- Road closures are caused by collisions
- Road closures could be inferred by seeing a common (low deviation) trip taking longer than usual for a given time